# Can the KNeighborsRegressor be used for time series forecasting?

In this notebook, we will learn how to make time-series predictions with the *KNeighborsRegressor* model, the regression variant of the *k*-NN algorithm.

We will work with the Airline Passengers dataset, which provides the monthly number of airline passengers from 1949-1960 for an airline company. This dataset is available on Kaggle and can be downloaded [here](https://www.kaggle.com/abhishekmamidi/air-passengers#AirPassengers.csv). 

## Load the data

In [ ]:
# Import libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# Load the data and create a datetime index
data = pd.read_csv("c4_AirPassengers.csv", index_col="Month", parse_dates=True)

# Rename columns for convenience
data.columns = ["Passengers"]
data.index.names = ["Date"]

# Print the shape of the dataframe
print("Data shape: {}".format(data.shape))

# Print a few samples
data.head()

## Exploratory data analysis

Let's have a first look at the data.

In [ ]:
# Register the converters

from pandas.plotting import register_matplotlib_converters

register_matplotlib_converters()

In [ ]:
# Plot the number of passengers
plt.figure(figsize=(8, 5))

font_dict = {"fontname": "Arial", "size": "14"}

plt.plot(data["Passengers"])
plt.xlabel("Year", fontdict=font_dict)
plt.ylabel("Monthly passengers (in thousands)", fontdict=font_dict)

plt.show()

In [ ]:
# Create a new colum: year
data["year"] = data.index.year

# Group by the 'year' and generate descriptive statistics
data.groupby("year").describe().T

Looking at the plot of the time-series and the table from above, we observe:  
* an increasing *trend*, as the *mean* number of passengers increases with time. For example, the mean number of passengers in 1949 is approximately 127. Six years later, in 1955, the average number of passengers has doubled (284) and, in 1960, is has almost quadrupled (476).
* a *seasonal pattern*, as summer months are more popular than winter months,
* that the annual variation in the data increases with time. In 1949, the number of passengers ranges from 104 to 148 (i.e., a total variation of 44 passengers), while in 1960, the total variation is 232 passengers!

We need to build forecasting models that take into account the trend, the seasonal pattern, and the increasing variation in the data.

In [ ]:
# Remove column: 'year'
data.drop("year", axis=1, inplace=True)

### Data transformation

The variation in the magnitude of the time-series with time is referred to as *non-stationarity in the variance*. Non-constant variance is common in time-series data, and we need to account for it before building our models, as this will help to improve predictions. 

One way of stabilizing the variance is by applying a logarithm transformation to the target. Log-transforming the number of passengers will help to "stabilize" the annual variation in the data to form a time-series that has a variance approximately constant over time.

However, there is no "one-size-fits-all" solution for all data; other types of transformations (inverse, square root, cubic root, etc.) may work better in other cases.

In [ ]:
# Log-transform 'Passengers'
data["Passengers"] = np.log(data["Passengers"])

# Print the head of the DataFrame
data.head()

In [ ]:
# Plot the log-transformed 'Passengers'

plt.figure(figsize=(8, 5))

plt.plot(data["Passengers"])
plt.xlabel("Year", fontdict=font_dict)
plt.ylabel("Log-Passengers", fontdict=font_dict)

plt.show()

We see that the log-transformation has helped to reduce the variation in size of the annual cycles in time.

## Fitting a first model

We will now to use *KNeighborsRegressor* to build a model that predicts the number of passengers in the next month. Before fitting our first model, let's have a look at the data once more.

In [ ]:
data.head()

### Feature engineering: the *Time* feature

At the moment, *Date* is expressed in a "year-month-day" format, which is not very convenient as it cannot be "processed" by the model. 

Let's create a new feature *Time* that is equal to the number of months since the first timestamp in the data (January 1, 1949).

In [ ]:
# New feature: 'Time'

data["Time"] = np.arange(0, data.shape[0])

# Print the head of the DataFrame
data[["Time", "Passengers"]].head()

### Training/test splitting for time-series data

We will now split into train, and test sets. We need to make sure that the test set covers a later period in time from the training set, to prevent any information about the future from "leaking" into the model during training. In this example, we will use the samples from 1949 to 1959 as our training set, and the last year in the data as our test set (1960).

Below, we define a function to split the data into training and test sets.

In [ ]:
# Define a function to split the data into training and test sets


def train_test_split(df, start_dt):

    """
    This function performs train/test splitting for time-series data.

    Arguments
    ---------
    - df        : the data (features & target)
    - start_df  : the starting date for the test set

    Outputs
    -------
    - X_tr,y_tr : X/y arrays for train set
    - X_te,y_te : X/y arrays for test data
    """

    # Train set
    X_tr = df[df.index < test_start_dt].drop("Passengers", axis=1).values
    y_tr = df[df.index < test_start_dt]["Passengers"]

    # Test set
    X_te = df[df.index >= test_start_dt].drop("Passengers", axis=1).values
    y_te = df[df.index >= test_start_dt]["Passengers"]

    print("Train set")
    print("---------")
    print("Features: {} Target: {}\n".format(X_tr.shape, y_tr.shape))

    print("Test set")
    print("--------")
    print("Features: {} Target: {}".format(X_te.shape, y_te.shape))

    return (X_tr, X_te, y_tr, y_te)

In [ ]:
# Split into train/test sets and create X/y arrays

# Set the starting date of the test set: Jan 1st, 1960
test_start_dt = "1960-01-01"

X_train, X_test, y_train, y_test = train_test_split(data, test_start_dt)

In [ ]:
# Plot the train/test sets

plt.figure(figsize=(8, 5))

plt.plot(y_train, label="Training data")
plt.plot(y_test, label="Test data")

plt.xlabel("Year", fontdict=font_dict)
plt.ylabel("Log-Passengers", fontdict=font_dict)
plt.legend()
plt.show()

We will now fit *KNeighborsRegressor* and evaluate its performance using the Mean Absolute Error (MAE). The target *Passengers* is log-transformed. This means that we need to invert the log-transformed *Passengers* back to their original scale before computing the error of the model. We can invert log-transformations using Numpy's exponential function (*numpy.exp*).

In [ ]:
# Mean Absolute error (MAE)
def MAE(y, y_pred):
    return np.mean(np.abs(y - y_pred))


# Invert log-transformation and compute model error
def model_error(y, y_pred):
    score = MAE(np.exp(y), np.exp(y_pred))  # Invert log-transformation and compute MAE
    return score

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

# Create and fit a KNeighborsRegressor model
knn = KNeighborsRegressor(
    n_neighbors=3, p=1, weights="distance"
)  # Hyperparameters: tuned with GridSearchCV (not shown here)
knn.fit(X_train, y_train)

# Compute predictions on train/test sets
y_pred_tr = knn.predict(X_train)
y_pred_te = knn.predict(X_test)

# Model performance
print("Model 1")
print("-------")
print()
print(
    "Train set - MAE score: {:.0f} passengers".format(model_error(y_train, y_pred_tr))
)
print("Test set  - MAE score: {:.0f} passengers".format(model_error(y_test, y_pred_te)))

In [ ]:
# Define a function to plot predictions


def plot_predictions(df, y_tr, y_te, y_pred_tr, y_pred_te, score_te):

    """
    This function plots the training/test data along with the predictions

    Arguments
    ---------
    - y_tr, y_te           : target (train/test sets)
    - y_pred_tr, y_pred_te : predictions (train/test sets)
    - score_te             : model error (test set)

    """

    font = {"fontname": "Arial", "size": "14"}
    font_title = {"fontname": "Arial", "size": "16"}

    plt.figure(figsize=(9, 6))

    # True values
    plt.plot(df.index[0 : len(y_tr)], np.exp(y_tr), label="train set", color="black")
    plt.plot(
        df.index[len(y_tr) : len(y_tr) + len(y_te)],
        np.exp(y_te),
        label="test set",
        color="grey",
    )

    # Predictions
    plt.plot(
        df.index[0 : len(y_tr)],
        np.exp(y_pred_tr),
        label="predictions train",
        color="blue",
    )
    plt.plot(
        df.index[len(y_tr) : len(y_tr) + len(y_te)],
        np.exp(y_pred_te),
        "-",
        label="predictions test",
        color="red",
    )

    plt.xlabel("Year", **font)
    plt.ylabel("Monthly passengers (in thousands)", **font)
    plt.title(
        "MAE score: {:.0f} passengers (test set)".format(score_te),
        fontsize=14,
        **font_title
    )

    plt.legend()
    plt.show()

In [ ]:
# Plot results

plot_predictions(
    data, y_train, y_test, y_pred_tr, y_pred_te, model_error(y_test, y_pred_te)
)

The predictions for the training set are good. However, for the test set, the model predicted a continuous line.

In the training data, the values of the *Time* feature range between 0 and 131, while in the test set they range between 132 and 143. The issue here is that *KNeighborsRegressor* cannot extrapolate to feature values outside the training set. As a result, for the test set, *KNeighborsRegressor* predicts the mean number of the passengers of the k-nearest neighbors from the training set, which are the last *3* timesteps in the training set.

## Fitting a second model

Let's see how we can improve our model. We will remove the *Time* feature, and we will add two new features: the *month* and the *year*. 

### Feature engineering: the *Year* and *Month* features

In [ ]:
# Create new features: month and year
data["Year"] = data.index.year
data["Month"] = data.index.month

# Remove column 'Time'
data.drop("Time", axis=1, inplace=True)

data[["Month", "Year", "Passengers"]].head()

In this second model, *Year* and *Month* are going to be the features and *Passengers* the target.

In [ ]:
# Split into train/set sets and create X/y arrays

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(data, test_start_dt)

In [ ]:
# Show a few samples from the training data

X_train_2[0:3]

In [ ]:
# Show a few samples from the test data

X_test_2[0:3]

Below we will fit a second *KNeighborsRegressor* model and evaluate its performance using the Mean Absolute Error (MAE).

In [ ]:
# Fit a second KNeighborsRegressor model

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipe = Pipeline(
    [
        ("scaler", StandardScaler()),  # Scaling improves performance
        ("knn", KNeighborsRegressor(n_neighbors=3, p=2, weights="distance")),
    ]
)  # Hyperparameters: tuned with GridSearchCV (not shown here)
pipe.fit(X_train_2, y_train_2)

# Compute predictions for train/test sets
y_pred_tr_2 = pipe.predict(X_train_2)
y_pred_te_2 = pipe.predict(X_test_2)

# Model performance
print("Model 2")
print("-------")
print()
print(
    "Train set - MAE score: {:.0f} passengers".format(
        model_error(y_train_2, y_pred_tr_2)
    )
)
print(
    "Test set  - MAE score: {:.0f} passengers".format(
        model_error(y_test_2, y_pred_te_2)
    )
)

In [ ]:
# Plot results

plot_predictions(
    data,
    y_train_2,
    y_test_2,
    y_pred_tr_2,
    y_pred_te_2,
    model_error(y_test_2, y_pred_te_2),
)

The predictions on the test set have improved, but we miss to capture the trend in the data. What this model is predicting at the moment is the mean number of passengers of the k-nearest neighbors in the training set. Unlike the first model, the k-nearest neighbors are now determined based on the month and year combination of the *k*-nearest neighbors in the training set, hence the improvement in the predictions.

## Fitting a third model  

Let's now see if we can further improve our model by eliminating the trend in the target. One way of removing the trend, i.e. the *non-stationarity in the mean*, is through *differencing*, which consists in computing the difference between consecutive observations. 

### Data preparation :: Differencing

Differencing can be achieved with the *pandas.DataFrame.diff* function. *Pandas.DataFrame.diff* first moves the time-series forward one position and then computes the difference with the original time-series (all in 1-step).

In [ ]:
# Create a copy of the data
data1 = data.copy()

# Column "Passengers" : apply differencing
data1["Passengers"] = data1["Passengers"].diff()

# Show a few samples from the data
data1[["Month", "Year", "Passengers"]].head()

Computing the first difference of the time-series resulted in a missing value in the first entry of *Passengers*. We need to remove missing values from the data before fiting our third model.

In [ ]:
# Remove rows with missing values
data1.dropna(inplace=True)

# Show a few samples from the data
data1[["Month", "Year", "Passengers"]].head()

In [ ]:
# Print the shape of the dataframe
print("Data shape: {}".format(data1.shape))

In [ ]:
# Plot the new feature 'Passengers'

plt.figure(figsize=(8, 5))

plt.plot(data1["Passengers"])

plt.xlabel("Year")
plt.ylabel("Differenced Log-Passengers", fontdict=font_dict)

plt.show()

Taking the first difference of the *Passengers* returned a time-series of the month-to-month changes in the number of passengers, as shown above. We notice that differencing has helped to stabilize the mean value of the target *Passengers*.

We are now ready to fit our third model; *Year* and *Month* are going to be the features and *Passengers* (differenced and log-transformed) the target.

In [ ]:
# Split into train/set sets and create X/y arrays

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(data1, test_start_dt)

In [ ]:
# Create and fit KNeighborsRegressor

# Here: better results without scaling
knn3 = KNeighborsRegressor(
    n_neighbors=2, p=1, weights="distance"
)  # Hyperparameters: tuned with GridSearchCV
knn3.fit(X_train_3, y_train_3)

# Compute predictions on train/test sets
y_pred_tr_3 = knn3.predict(X_train_3)
y_pred_te_3 = knn3.predict(X_test_3)

### Invert differencing

Now, the target and the predictions are expressed as the month-to-month change in the number of passengers. We need to invert differencing to turn the target and the predictions back to their original scale before computing the error of the model.

Differencing can be inverted using Numpy's cumulative sum function ([numpy.cumsum](https://numpy.org/doc/stable/reference/generated/numpy.cumsum.html)). However, differencing threw away the first entry of the target *Passengers*. So, before calling the *cumsum* function, we need to put this value back together with the predictions on the train and test sets. 

In [ ]:
# Define function to invert differencing


def invert_differencing(y_tr, y_te, y0, len_train):

    """
    This function inverts differencing of time-series data and
    re-splits the data into training/test sets.

    It takes the following arguments:
    - y_tr      : predictions on the train data (differenced)
    - y_te      : predictions on the test data (differenced)
    - y0        : the first entry of the target (before differencing)
    - len_train : number of samples in the training set (before differencing)

    Output:
    - y_train   : predictions of the train data (original scale)
    - y_test    : predictions of the test data (original scale)
    """

    y_tr_te_inv = np.r_[y0, y_tr, y_te].cumsum()  # Invert differencing
    y_train, y_test = (
        y_tr_te_inv[0:len_train],
        y_tr_te_inv[len_train:],
    )  # Train/test split

    return (y_train, y_test)

In [ ]:
# Retrieve the first entry of the target which was thrown away after differencing
y0 = data["Passengers"][0]

# Define the number of samples in the train data (before differencing)
len_train = len(y_train)

# Target: Invert differencing
y_train_3, y_test_3 = invert_differencing(y_train_3, y_test_3, y0, len_train)

# Predictions: Invert differencing
y_pred_tr_3, y_pred_te_3 = invert_differencing(y_pred_tr_3, y_pred_te_3, y0, len_train)

In [ ]:
# Model performance

print("Model 3")
print("-------")
print()
print(
    "Train set - MAE score: {:.0f} passengers".format(
        model_error(y_train_3, y_pred_tr_3)
    )
)
print(
    "Test set  - MAE score: {:.0f} passengers".format(
        model_error(y_test_3, y_pred_te_3)
    )
)

In [ ]:
# Plot results

plot_predictions(
    data,
    y_train_3,
    y_test_3,
    y_pred_tr_3,
    y_pred_te_3,
    model_error(y_test_3, y_pred_te_3),
)

Now we have a model that captures the periodic behavior and the trend in the number of airline passengers. This way, we managed to further improve model performance on the test set! 

In the last part of this exercise, we will introduce another type of feature that is commonly used in time-series prediction: *lag features*.

## Fitting a fourth model  

### Feature engineering :: Lag features

Time-series prediction models are often constructed using *lag features* as a means of incorporating information about the past into the model. Let's begin by discussing *how* to build *lag features*. 

If the observation at time t is $y_t$, then, the observation at the previous time-step is $y_{t-1}$, at the previous two time-steps $y_{t-2}$, and so on. We say that $y_{t-1}$ is *lagged* by one period, $y_{t-2}$ by two periods, etc. 

We can create *lag features* with the *pandas.DataFrame.shift* function using the *periods* keyword. Setting *periods=1* means that the time-series will be shifted forward one position and so on. Let's see how this works with an example.

*Note*: We will be working with the *detrended* and *log-transformed* *Passengers* column.

In [ ]:
# Create a copy of the data
data2 = data1[["Month", "Year", "Passengers"]].copy()

# Show a few samples
data2.head()

In [ ]:
# EXAMPLE
# Create lag features: t-1 and t-2
lags = pd.concat(
    [
        data2["Passengers"].shift(periods=2),
        data2["Passengers"].shift(periods=1),
        data2["Passengers"],
    ],
    axis=1,
)
lags.columns = ["t-2", "t-1", "t"]

# Show a few samples
lags.head()

The column t corresponds to the (detrended and log-transformed) *Passengers* column. *t-1* corresponds to the *Passengers* column shifted forward by one period, and the *t-2* shifted forward by two periods. 

Shifting the time-series has resulted in missing values at the beginning of the lagged features, which we will remove below with the *dropna* method.

In [ ]:
# Remove rows with missing values
lags.dropna(inplace=True)

# Show a few samples
lags.head()

So, we saw *how* to construct *lag features*. Let's now see how to select *meaningful* lag features. 

*Pandas.plotting.lag_plot* is a convenient tool for creating *lag plots* for time-series. Below, we use it to plot the *Passengers* column (*detrended* and *log-transformed*) against lagged features. Each graph shows $y_t$ plotted against $y_{t−lag}$, where *lag* ranges from 1 to 12.

In [ ]:
# Lag plot of "Passengers"

fig, axes = plt.subplots(2, 6, figsize=(18, 6))

for lag, ax in zip(np.arange(1, 13), axes.ravel()):
    pd.plotting.lag_plot(data2["Passengers"], lag=lag, ax=ax, alpha=0.6)

plt.tight_layout()

We see that *Passengers* is correlated with lag 12; this is due to the seasonality in the data. The rest of the lags show no, or weak correlation with *Passengers*.

In our last model, we will use *lag 12* together with the *Month* and *Year* features.

In [ ]:
# New feature: lag 12
data2["t-12"] = data2["Passengers"].shift(12)

# Re-order columns for convenience
data2 = data2[["t-12", "Month", "Year", "Passengers"]].copy()

# Show a few samples
data2.head(15)

In [ ]:
# Remove rows with missing values
data2.dropna(inplace=True)

# Show a few samples from the data
data2.head()

In [ ]:
# Print the shape of the dataframe
print("Data shape: {}".format(data2.shape))

We are now ready to fit our fourth model; *t-12*, *Year* and *Month* are going to be the features and *Passengers* (differenced and log-transformed) the target.

In [ ]:
# Split into train/set sets and create X/y arrays

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(data2, test_start_dt)

In [ ]:
# Create and fit KNeighborsRegressor

pipe4 = Pipeline(
    [
        ("scaler", StandardScaler()),  # Scaling improves performance
        ("knn", KNeighborsRegressor(n_neighbors=3, p=1, weights="distance")),
    ]
)  # Hyperparameters: tuned with GridSearchCV
pipe4.fit(X_train_4, y_train_4)

# Compute predictions for train/test sets
y_pred_tr_4 = pipe4.predict(X_train_4)
y_pred_te_4 = pipe4.predict(X_test_4)

As in the third model, the target and the predictions are expressed as the month-to-month change in the number of passengers. We will invert differencing using the *invert_differencing* function from above.

However, we need to pay attention to the following:
* differencing threw away the first entry of the target
* shifting the time-series forward by 12 periods threw away the next 12 observations in the target.

We need to put these 13 (1 + 12) values back before calling the *cumsum* function.

In [ ]:
# Retrieve thrown away entries
y0 = np.r_[data["Passengers"][0], data1["Passengers"][0:12]]  # length: 13

# Define the number of samples in the train data (before differencing and lagging)
len_train = len(y_train)

# Target: Invert differencing
y_train_4, y_test_4 = invert_differencing(y_train_4, y_test_4, y0, len_train)

# Predictions: Invert differencing
y_pred_tr_4, y_pred_te_4 = invert_differencing(y_pred_tr_4, y_pred_te_4, y0, len_train)

In [ ]:
# Model performance
print("Model 4")
print("-------")
print()
print(
    "Train set - MAE score: {:.0f} passengers".format(
        model_error(y_train_4, y_pred_tr_4)
    )
)
print(
    "Test set  - MAE score: {:.0f} passengers".format(
        model_error(y_test_4, y_pred_te_4)
    )
)

In [ ]:
# Plot results

plot_predictions(
    data,
    y_train_4,
    y_test_4,
    y_pred_tr_4,
    y_pred_te_4,
    model_error(y_test_4, y_pred_te_4),
)

## Final model comparison

In [ ]:
# Final comparison

font = {"fontname": "Arial", "size": "14"}
font_title = {"fontname": "Arial", "size": "16"}

plt.figure(figsize=(10, 6))

labels = ["Model 1", "Model 2", "Model 3", "Model 4"]

mae_values = [
    model_error(y_test, y_pred_te),  # MAE :  model 1
    model_error(y_test_2, y_pred_te_2),  # MAE :  model 2
    model_error(y_test_3, y_pred_te_3),  # MAE :  model 3
    model_error(y_test_4, y_pred_te_4),  # MAE :  model 4
]

xcor = np.arange(len(mae_values))
plt.bar(xcor, mae_values, color="g", edgecolor="black")
plt.xticks(xcor, labels, **font)
plt.ylabel("MAE", **font)
plt.title("Model comparison", **font_title)
plt.show()

## Summary

* Creating *meaningful* features is essential for building good machine learning models. In this exercise, we explored *date-time* and *lag* features and achieved good model performances.

* In time-series data, transformations such as applying the logarithm and differencing can help to stabilize the *non-stationarity* in the variance and the mean, respectively, and improve model performance. However, there is no "one-size-fits-all" solution; other types of transformations (inverse, square root, cubic root, etc.) may work better depending on the time-series data and the model used.

* Transformations need to be inverted to turn the observations back to their original scale. Log-transformations (natural base) can be reversed by applying the exponential function. Differencing can be reversed using the cumulative sum (*cumsum*) function.

* The *KNeighborsRegressor* model is not the only Machine Learning model that can be used for time-series prediction! Feel free to experiment time-series forecasting with other models: LinearRegression, RandomForestRegressor, and neural networks!